# pip


In [1]:
!pip install supervision
!pip install opencv-python
!pip install deep-sort-realtime
!pip install torch torchvision torchaudio
!pip install ultralytics
!pip install sympy==1.12
!pip install opencv-contrib-python

  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
  Using cached numpy-2.1.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.1.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.1 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.1.1 which is incompatible.
  Using cached sympy-1.12-py3-none-a

In [6]:
!pip install --upgrade numpy
!pip install --upgrade scipy
!pip install --upgrade --force-reinstall numpy
!pip uninstall numpy
!pip install --upgrade pip setuptools wheel
!pip install -U opencv-python opencv-contrib-python ultralytics deep-sort-realtime
!pip install --upgrade pip setuptools wheel
!pip install sympy==1.12
!pip install numpy==1.26.4
!pip install opencv-python opencv-python-headless mediapipe scikit-learn tensorflow

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.105 requires numpy<=2.1.1,>=1.23.0, but you have numpy 2.2.4 which is incompatible.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using

ERROR: Operation cancelled by user
^C
^C
^C
^C


# data video

In [3]:
import os


os.environ["KAGGLEHUB_CACHE"] = "/content/data"

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shreyamainkar/football-soccer-videos-dataset")

print("Path to dataset files:", path)

100%|██████████| 7.74G/7.74G [01:23<00:00, 99.3MB/s]

Extracting files...


Path to dataset files: /content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2


# Model

In [2]:
import numpy as np
from ultralytics import YOLO
from sklearn.cluster import KMeans
from scipy.spatial import ConvexHull
from collections import defaultdict, deque
from sklearn.metrics import pairwise_distances
import pickle
import os
import pandas as pd
import supervision as sv
import cv2


class TacticalSoccerTracker:
    def __init__(self, video_path, output_path):
        self.input_video_path = video_path
        self.output_video_path = output_path
        self.model = YOLO("yolov8n.pt")
        self.tracker = sv.ByteTrack()

        # speed smoothing
        self.tracking_history = defaultdict(lambda: {
            'positions': deque(maxlen=30),
            'speeds': deque(maxlen=30),
            'speed_smooth': 0,
            'distance': 0
        })

        self.player_assignments = {}
        self.team_colors = {1: (0, 100, 255), 2: (0, 200, 200)}  # Blue and Cyan

        # Visual settings
        self.visual_settings = {
            'team_colors': {
                1: (0, 100, 255),    # Blue
                2: (0, 200, 200),    # Cyan
                'ball': (0, 0, 255)   # Red
            },
            'speed_colors': {
                'slow': (0, 255, 0),    # Green
                'medium': (0, 255, 255),# Yellow
                'fast': (0, 165, 255),  # Orange
                'very_fast': (0, 0, 255)# Red
            },
            'panel': {
                'bg_color': (40, 40, 40),
                'text_color': (255, 255, 255),
                'highlight_color': (255, 255, 0),
                'font': cv2.FONT_HERSHEY_SIMPLEX,
                'font_scale': 0.6,
                'thickness': 1,
                'spacing': 35,
                'panel_width': 500
            }
        }

        self.teams = {
            1: {
                'color': self.visual_settings['team_colors'][1],
                'stats': {
                    'possession': 50,
                    'avg_speed': 0,
                    'max_speed': 0,
                    'speed_history': deque(maxlen=100),
                    'top_players': []
                }
            },
            2: {
                'color': self.visual_settings['team_colors'][2],
                'stats': {
                    'possession': 50,
                    'avg_speed': 0,
                    'max_speed': 0,
                    'speed_history': deque(maxlen=100),
                    'top_players': []
                }
            }
        }

        self.frame_id = 0
        self.team_colors_identified = False
        self.ball_team_frames = {1: 0, 2: 0}
        self.fps = 30
        self.field_scale = (105, 68)

    def setup_video(self):
        self.capture = cv2.VideoCapture(self.input_video_path)
        if not self.capture.isOpened():
            raise ValueError(f"Could not open video: {self.input_video_path}")

        self.frame_width = int(self.capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(self.capture.get(cv2.CAP_PROP_FPS))

        self.output_width = self.frame_width + self.visual_settings['panel']['panel_width']
        self.output_height = self.frame_height

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        self.writer = cv2.VideoWriter(
            self.output_video_path, fourcc, self.fps,
            (self.output_width, self.output_height)
        )

    def get_foot_position(self, bbox):
        x1, y1, x2, y2 = bbox
        return int((x1 + x2) / 2), int(y2)

    def get_center(self, bbox):
        x1, y1, x2, y2 = bbox
        return int((x1 + x2) / 2), int((y1 + y2) / 2)

    def measure_distance(self, p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    def calculate_speed(self, player_id, current_pos):
        history = self.tracking_history[player_id]
        if len(history['positions']) > 0:
            prev_pos = history['positions'][-1]
            distance = self.measure_distance(prev_pos, current_pos)
            instant_speed = (distance / 10) * self.fps * 3.6  # km/h
            smoothed_speed = 0.7 * history['speed_smooth'] + 0.3 * instant_speed if history['speed_smooth'] != 0 else instant_speed
            history['speed_smooth'] = smoothed_speed
            history['speeds'].append(smoothed_speed)
            history['distance'] += distance
            return smoothed_speed
        return 0

    def get_speed_category(self, speed):
        if speed < 8: return 'slow'
        elif speed < 15: return 'medium'
        elif speed < 22: return 'fast'
        else: return 'very_fast'

    def assign_teams(self, frame, player_boxes):
        if len(player_boxes) < 6:
            return False

        jersey_colors = []
        player_ids = []

        for box in player_boxes:
            x1, y1, x2, y2 = box['bbox']
            player_id = box['id']
            if player_id is None:
                continue

            jersey_region = frame[y1:y1+int((y2-y1)*0.6), x1:x2]
            if jersey_region.size == 0:
                continue

            pixels = jersey_region.reshape(-1, 3).astype(np.float32)
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.5)
            _, labels, centers = cv2.kmeans(pixels, 1, None, criteria, 10, cv2.KMEANS_PP_CENTERS)

            dominant_color = centers[0]
            jersey_colors.append(dominant_color)
            player_ids.append(player_id)

        if len(jersey_colors) < 2:
            return False

        jersey_colors_np = np.array(jersey_colors)
        jersey_colors_norm = jersey_colors_np / 255.0

        kmeans = KMeans(n_clusters=2, random_state=0).fit(jersey_colors_norm)
        labels = kmeans.labels_

        for pid, label in zip(player_ids, labels):
            self.player_assignments[pid] = label + 1  # 1 or 2

        self.team_colors_identified = True
        return True


    def get_player_team(self, player_id):
        if player_id is None:
            return 1
        if player_id in self.player_assignments:
            return self.player_assignments[player_id]
        return 1 if player_id % 2 == 0 else 2


    def draw_player_marker(self, frame, bbox, team_id, player_id, speed):
        color = self.team_colors[team_id]
        x1, y1, x2, y2 = bbox

        # bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # speed indicator (colored circle)
        speed_category = self.get_speed_category(speed)
        speed_color = self.visual_settings['speed_colors'][speed_category]
        cv2.circle(frame, (x1 + 15, y1 - 15), 8, speed_color, -1)

        # player ID and speed
        info_text = f"{player_id}: {speed:.1f}km/h"
        text_size = cv2.getTextSize(info_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
        text_x = max(x1, 0)
        text_y = max(y1 - 5, text_size[1] + 5)
        cv2.putText(frame, info_text, (text_x, text_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    def update_team_statistics(self, current_players, ball_position):
        team_speeds = {1: [], 2: []}
        team_players = {1: {}, 2: {}}

        # Update player and team stats
        for player_id, player in current_players.items():
            team_id = self.get_player_team(player_id)
            speed = player.get('speed', 0)
            team_speeds[team_id].append(speed)

            # Track individual player stats
            if player_id not in team_players[team_id]:
                team_players[team_id][player_id] = {
                    'speed_history': deque(maxlen=50),
                    'max_speed': 0,
                    'avg_speed': 0
                }

            player_stats = team_players[team_id][player_id]
            player_stats['speed_history'].append(speed)
            player_stats['max_speed'] = max(player_stats['max_speed'], speed)
            if player_stats['speed_history']:
                player_stats['avg_speed'] = np.mean(player_stats['speed_history'])

            self.tracking_history[player_id]['positions'].append(player['position'])

        # Update team stats
        for team_id in [1, 2]:
            if team_speeds[team_id]:
                self.teams[team_id]['stats']['avg_speed'] = np.mean(team_speeds[team_id])
                self.teams[team_id]['stats']['max_speed'] = max(team_speeds[team_id])
                self.teams[team_id]['stats']['speed_history'].append(np.mean(team_speeds[team_id]))

                # Update top players
                players = [(pid, stats['avg_speed']) for pid, stats in team_players[team_id].items()]
                top_players = sorted(players, key=lambda x: x[1], reverse=True)[:3]  # Top 3 players
                self.teams[team_id]['stats']['top_players'] = top_players

        # Update ball possession
        if ball_position:
            closest_team = None
            min_dist = float('inf')
            for player_id, player in current_players.items():
                team_id = self.get_player_team(player_id)
                dist = self.measure_distance(ball_position, player['position'])
                threshold = 100 * (1 + (player.get('speed', 0) / 50))
                if dist < threshold and dist < min_dist:
                    min_dist = dist
                    closest_team = team_id

            if closest_team:
                self.ball_team_frames[closest_team] += 1

        # Update possession percentages
        total_frames = sum(self.ball_team_frames.values())
        if total_frames > 0:
            for team_id in [1, 2]:
                self.teams[team_id]['stats']['possession'] = (
                    self.ball_team_frames[team_id] / total_frames * 100)

    def draw_speed_graph(self, frame, team_id, x, y, width=200, height=80):

        speeds = list(self.teams[team_id]['stats']['speed_history'])
        if len(speeds) < 2:
            return

        # Normalize speeds to fit graph height
        max_speed = max(speeds) if max(speeds) > 0 else 30
        normalized = [int((s / max_speed) * height) for s in speeds]

        # graph background
        cv2.rectangle(frame, (x, y), (x + width, y + height), (60, 60, 60), -1)

        # grid lines
        for i in range(0, height + 1, height//4):
            cv2.line(frame, (x, y + i), (x + width, y + i), (100, 100, 100), 1)

        # speed line
        points = []
        for i, val in enumerate(normalized):
            x_pos = x + int((i / len(normalized)) * width)
            y_pos = y + height - val
            points.append((x_pos, y_pos))

        if len(points) > 1:
            cv2.polylines(frame, [np.array(points)], False, self.teams[team_id]['color'], 2)

        # current speed label
        current_speed = speeds[-1] if speeds else 0
        cv2.putText(frame, f"{current_speed:.1f} km/h",
                   (x + 10, y + 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    def draw_top_players(self, frame, team_id, x, y):

        top_players = self.teams[team_id]['stats']['top_players']
        if not top_players:
            return

        cv2.putText(frame, "Top Players:",
                   (x, y),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        for i, (player_id, avg_speed) in enumerate(top_players):
            y_pos = y + 25 + i * 25
            cv2.putText(frame, f"P{player_id}: {avg_speed:.1f} km/h",
                       (x + 10, y_pos),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.teams[team_id]['color'], 1)

    def draw_statistics_panel(self, frame):
        # right panel
        panel_width = self.visual_settings['panel']['panel_width']
        panel = np.zeros((self.output_height, panel_width, 3), dtype=np.uint8)
        panel[:] = self.visual_settings['panel']['bg_color']

        # Combine with main frame
        combined_frame = np.hstack((frame, panel))

        # header
        header_y = 40
        cv2.putText(combined_frame, "Tactical Analysis",
                   (self.frame_width + 20, header_y),
                   self.visual_settings['panel']['font'],
                   0.7, (255, 255, 255), 2, cv2.LINE_AA)

        #  team stats sections
        section_height = 250  # Increased section height
        section_spacing = 30  # Space between team sections

        for i, team_id in enumerate([1, 2]):
            team = self.teams[team_id]
            y_offset = header_y + 50 + i * (section_height + section_spacing)

            # Team header
            cv2.putText(combined_frame, f"Team {team_id} Performance",
                       (self.frame_width + 20, y_offset),
                       self.visual_settings['panel']['font'],
                       0.6, team['color'], 1, cv2.LINE_AA)

            # Key stats with proper line spacing (30px between lines)
            stats_y = y_offset + 30
            line_spacing = 30

            cv2.putText(combined_frame,
                       f"Possession: {team['stats']['possession']:.1f}%",
                       (self.frame_width + 20, stats_y),
                       self.visual_settings['panel']['font'],
                       0.5, self.visual_settings['panel']['text_color'], 1, cv2.LINE_AA)

            cv2.putText(combined_frame,
                       f"Avg Speed: {team['stats']['avg_speed']:.1f} km/h",
                       (self.frame_width + 20, stats_y + line_spacing),
                       self.visual_settings['panel']['font'],
                       0.5, self.visual_settings['panel']['text_color'], 1, cv2.LINE_AA)

            cv2.putText(combined_frame,
                       f"Max Speed: {team['stats']['max_speed']:.1f} km/h",
                       (self.frame_width + 20, stats_y + line_spacing * 2),
                       self.visual_settings['panel']['font'],
                       0.5, self.visual_settings['panel']['text_color'], 1, cv2.LINE_AA)

            # Speed trend graph
            graph_y = stats_y + line_spacing * 3 + 10
            self.draw_speed_graph(combined_frame, team_id, self.frame_width + 20, graph_y)


        # Speed legend
        legend_y = self.output_height - 130
        cv2.putText(combined_frame, "Speed Categories:",
                   (self.frame_width + 20, legend_y),
                   self.visual_settings['panel']['font'],
                   0.5, (255, 255, 255), 1, cv2.LINE_AA)

        categories = [
            ("Slow (<8 km/h)", 'slow'),
            ("Medium (8-15 km/h)", 'medium'),
            ("Fast (15-22 km/h)", 'fast'),
            ("Very Fast (>22 km/h)", 'very_fast')
        ]

        for i, (label, cat) in enumerate(categories):
            y_pos = legend_y + 25 + i * 25
            cv2.circle(combined_frame,
                      (self.frame_width + 20, y_pos),
                      6, self.visual_settings['speed_colors'][cat], -1)
            cv2.putText(combined_frame, label,
                       (self.frame_width + 40, y_pos + 5),
                       self.visual_settings['panel']['font'],
                       0.45, (255, 255, 255), 1, cv2.LINE_AA)

        return combined_frame

    def process_video(self):
        self.setup_video()

        while self.capture.isOpened():
            ret, frame = self.capture.read()
            if not ret:
                break

            # Detect and track objects
            results = self.model.track(
                frame,
                persist=True,
                classes=[0, 32],  # Players and ball
                conf=0.6,
                iou=0.5,
                imgsz=640,
                tracker="bytetrack.yaml"
            )

            current_players = {}
            ball_position = None

            if results and results[0].boxes is not None:
                for box in results[0].boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    cls_id = int(box.cls[0])
                    track_id = int(box.id[0]) if box.id is not None else None

                    if conf < 0.6:
                        continue

                    if cls_id == 0:  # Player
                        position = self.get_foot_position((x1, y1, x2, y2))
                        speed = self.calculate_speed(track_id, position)
                        current_players[track_id] = {
                            'bbox': (x1, y1, x2, y2),
                            'position': position,
                            'speed': speed
                        }
                        self.tracking_history[track_id]['positions'].append(position)

                    elif cls_id == 32:  # Ball
                        ball_position = self.get_center((x1, y1, x2, y2))

            # Assign teams on first frame with enough players
            if not self.team_colors_identified and len(current_players) >= 6:
                player_boxes = [{'bbox': player['bbox'], 'id': pid} for pid, player in current_players.items()]
                self.assign_teams(frame, player_boxes)

            # players
            for player_id, player in current_players.items():
                if player_id is not None:
                    team_id = self.get_player_team(player_id)
                    self.draw_player_marker(frame, player['bbox'], team_id, player_id, player['speed'])

            # ball
            if ball_position:
                cv2.circle(frame, ball_position, 10, (0,0,0), -1)
                cv2.circle(frame, ball_position, 8, self.visual_settings['team_colors']['ball'], -1)
                cv2.circle(frame, ball_position, 3, (255,255,255), -1)

            # Update and draw statistics
            self.update_team_statistics(current_players, ball_position)
            output_frame = self.draw_statistics_panel(frame)

            # Write processed frame
            self.writer.write(output_frame)
            self.frame_id += 1

            if self.frame_id % 100 == 0:
                print(f"Processed frame {self.frame_id}")

        self.capture.release()
        self.writer.release()
        print(f"Analysis complete. Output saved to {self.output_video_path}")


# test on videos

In [3]:

if __name__ == "__main__":
    input_video = "/content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2/104.mp4"
    output_video = "tactical__output.mp4"
    tracker = TacticalSoccerTracker(input_video, output_video)
    tracker.process_video()

Streaming output truncated to the last 5000 lines.

0: 384x640 12 persons, 173.7ms
Speed: 3.4ms preprocess, 173.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 161.9ms
Speed: 5.8ms preprocess, 161.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 166.7ms
Speed: 5.4ms preprocess, 166.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 165.5ms
Speed: 3.4ms preprocess, 165.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 153.1ms
Speed: 5.1ms preprocess, 153.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 152.4ms
Speed: 4.5ms preprocess, 152.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 155.0ms
Speed: 3.2ms preprocess, 155.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 153.5ms
Speed: 3.

In [5]:

if __name__ == "__main__":
    input_video = "/content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2/118.mp4"
    output_video = "tactical__output2.mp4"
    tracker = TacticalSoccerTracker(input_video, output_video)
    tracker.process_video()

Streaming output truncated to the last 5000 lines.
0: 384x640 8 persons, 154.8ms
Speed: 3.3ms preprocess, 154.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 152.8ms
Speed: 4.5ms preprocess, 152.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 155.7ms
Speed: 3.3ms preprocess, 155.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 173.3ms
Speed: 3.3ms preprocess, 173.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 178.1ms
Speed: 4.3ms preprocess, 178.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 171.8ms
Speed: 7.2ms preprocess, 171.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 168.6ms
Speed: 3.6ms preprocess, 168.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 169.1ms
Speed: 3.8ms pre

In [6]:

if __name__ == "__main__":
    input_video = "/content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2/12.mp4"
    output_video = "tactical__output3.mp4"
    tracker = TacticalSoccerTracker(input_video, output_video)
    tracker.process_video()

Streaming output truncated to the last 5000 lines.
0: 384x640 3 persons, 163.0ms
Speed: 5.4ms preprocess, 163.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 148.7ms
Speed: 5.0ms preprocess, 148.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 145.5ms
Speed: 3.3ms preprocess, 145.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 149.1ms
Speed: 3.0ms preprocess, 149.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 154.8ms
Speed: 4.8ms preprocess, 154.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 168.6ms
Speed: 4.1ms preprocess, 168.6ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 162.9ms
Speed: 5.0ms preprocess, 162.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 163.3ms
Speed: 3.6ms pre

In [7]:

if __name__ == "__main__":
    input_video = "/content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2/129.mp4"
    output_video = "tactical__output4.mp4"
    tracker = TacticalSoccerTracker(input_video, output_video)
    tracker.process_video()

Streaming output truncated to the last 5000 lines.

0: 384x640 9 persons, 154.1ms
Speed: 4.1ms preprocess, 154.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 167.9ms
Speed: 3.7ms preprocess, 167.9ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 176.8ms
Speed: 3.5ms preprocess, 176.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 155.7ms
Speed: 3.0ms preprocess, 155.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 150.1ms
Speed: 3.1ms preprocess, 150.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 151.7ms
Speed: 3.1ms preprocess, 151.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 154.8ms
Speed: 2.9ms preprocess, 154.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 168.3ms
Speed: 3.6ms prepr

In [8]:

if __name__ == "__main__":
    input_video = "/content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2/3.mp4"
    output_video = "tactical__output5.mp4"
    tracker = TacticalSoccerTracker(input_video, output_video)
    tracker.process_video()

Streaming output truncated to the last 5000 lines.
0: 384x640 6 persons, 149.1ms
Speed: 4.8ms preprocess, 149.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 153.0ms
Speed: 3.8ms preprocess, 153.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 171.9ms
Speed: 6.4ms preprocess, 171.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 172.9ms
Speed: 3.7ms preprocess, 172.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 166.8ms
Speed: 3.7ms preprocess, 166.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 161.4ms
Speed: 5.3ms preprocess, 161.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 8000

0: 384x640 6 persons, 154.1ms
Speed: 3.5ms preprocess, 154.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 152

In [9]:

if __name__ == "__main__":
    input_video = "/content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2/45.mp4"
    output_video = "tactical__output6.mp4"
    tracker = TacticalSoccerTracker(input_video, output_video)
    tracker.process_video()

Streaming output truncated to the last 5000 lines.

0: 384x640 1 person, 250.0ms
Speed: 4.3ms preprocess, 250.0ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 269.8ms
Speed: 3.7ms preprocess, 269.8ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 263.7ms
Speed: 3.7ms preprocess, 263.7ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 246.1ms
Speed: 3.6ms preprocess, 246.1ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 279.3ms
Speed: 3.8ms preprocess, 279.3ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 273.1ms
Speed: 5.8ms preprocess, 273.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 248.1ms
Speed: 3.2ms preprocess, 248.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 256.0ms
Speed: 3.1ms preprocess

In [10]:

if __name__ == "__main__":
    input_video = "/content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2/82.mp4"
    output_video = "tactical__output7.mp4"
    tracker = TacticalSoccerTracker(input_video, output_video)
    tracker.process_video()

Streaming output truncated to the last 5000 lines.
0: 384x640 8 persons, 183.6ms
Speed: 3.5ms preprocess, 183.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 1 sports ball, 164.2ms
Speed: 3.3ms preprocess, 164.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 sports ball, 168.0ms
Speed: 3.4ms preprocess, 168.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 161.9ms
Speed: 3.9ms preprocess, 161.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 166.6ms
Speed: 4.2ms preprocess, 166.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 173.7ms
Speed: 4.8ms preprocess, 173.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 168.0ms
Speed: 3.7ms preprocess, 168.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 pe

In [11]:

if __name__ == "__main__":
    input_video = "/content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2/99.mp4"
    output_video = "tactical__output8.mp4"
    tracker = TacticalSoccerTracker(input_video, output_video)
    tracker.process_video()

Streaming output truncated to the last 5000 lines.

0: 384x640 (no detections), 161.2ms
Speed: 3.4ms preprocess, 161.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 180.8ms
Speed: 4.1ms preprocess, 180.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 192.2ms
Speed: 6.3ms preprocess, 192.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 265.0ms
Speed: 6.1ms preprocess, 265.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 232.4ms
Speed: 3.4ms preprocess, 232.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 285.5ms
Speed: 7.4ms preprocess, 285.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 246.5ms
Speed: 7.5ms preprocess, 246.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 

In [12]:

if __name__ == "__main__":
    input_video = "/content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2/22.mp4"
    output_video = "tactical__output9.mp4"
    tracker = TacticalSoccerTracker(input_video, output_video)
    tracker.process_video()

Streaming output truncated to the last 5000 lines.

0: 384x640 5 persons, 157.9ms
Speed: 4.7ms preprocess, 157.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 154.4ms
Speed: 3.0ms preprocess, 154.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 153.3ms
Speed: 2.9ms preprocess, 153.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 159.0ms
Speed: 3.0ms preprocess, 159.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 171.5ms
Speed: 3.1ms preprocess, 171.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 158.1ms
Speed: 3.2ms preprocess, 158.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 154.6ms
Speed: 3.0ms preprocess, 154.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 167.4ms
Speed: 2.9ms pr

In [ ]:

if __name__ == "__main__":
    input_video = "/content/data/datasets/shreyamainkar/football-soccer-videos-dataset/versions/2/160.mp4"
    output_video = "tactical__output10.mp4"
    tracker = TacticalSoccerTracker(input_video, output_video)
    tracker.process_video()


0: 384x640 (no detections), 148.6ms
Speed: 2.6ms preprocess, 148.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 149.2ms
Speed: 3.0ms preprocess, 149.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 153.5ms
Speed: 3.0ms preprocess, 153.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 153.9ms
Speed: 2.9ms preprocess, 153.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 154.5ms
Speed: 3.0ms preprocess, 154.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 165.0ms
Speed: 2.9ms preprocess, 165.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 150.5ms
Speed: 3.0ms preprocess, 150.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 157.1ms
Speed: 4.1ms prepr